In [1]:
import pandas as pd
import glob
import time

In [2]:
sales=pd.read_csv("sales.csv",dtype={"Zip Code":str})
sales["Address"]=sales["Address"].map(lambda x:str.split(x,"#")[0])
sales["address"]=sales["Address"]+", Washington, DC "+sales["Zip Code"]

In [3]:
gh=pd.read_csv("good high school.csv",dtype={"Zip Code":str})
gh["address"]=gh["Zip Code"]

In [4]:
gh

,Unnamed: 0,School Code,School Name,Number of Test-takers,Evidence-Based Reading and Writing Average,Math Average,Total Average,Zip Code,Subgroup,# Grads,% Grad,Performance of Graduation,Performance of SAT,Performance,Ranking,address
0,0,402,Benjamin Banneker HS,221,563,546,1109.0,20001,All,103,1.00,5,5,5.0,5,20001
1,13,466,School Without Walls HS,242,646,626,1272.0,20037,All,154,0.99,5,5,5.0,5,20037
2,8,458,McKinley Technology HS,282,508,497,1005.0,20002,All,137,0.96,5,4,4.5,5,20002
3,17,478,"Phelps Architecture, Construction and Engineer...",112,451,424,875.0,20002,All,65,0.98,5,4,4.5,5,20002
4,15,471,Duke Ellington School of the Arts,230,500,454,955.0,20007,All,118,1.00,5,4,4.5,5,20007


In [5]:
gh["address"][0]="800 Euclid St NW, Washington, DC 20001"
gh["address"][1]="151 T St NE, Washington, DC 20002"
gh["address"][2]="3950 Chesapeake St NW, Washington, DC 20016"
gh["address"][3]="2130 G St NW, Washington, DC 20037"
gh["address"][4]="3500 R St NW, Washington, DC 20007"
gh["address"][5]="704 26th St NE, Washington, DC 20002"
gh["address"][6]="3101 16th St NW, Washington, DC 20010"

C:\Users\yz402\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\yz402\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\yz402\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we c

In [6]:
from geopy.geocoders import Nominatim
from geopy import distance

In [8]:
gh["address"]

0         800 Euclid St NW, Washington, DC 20001
1              151 T St NE, Washington, DC 20002
2    3950 Chesapeake St NW, Washington, DC 20016
3             2130 G St NW, Washington, DC 20037
4             3500 R St NW, Washington, DC 20007
5           704 26th St NE, Washington, DC 20002
6          3101 16th St NW, Washington, DC 20010
Name: address, dtype: object

In [7]:
geolocator = Nominatim(user_agent="high school")
loca=[]
for i in range(len(gh)):
    location = geolocator.geocode(gh["address"][i])
    loca.append((location.latitude, location.longitude))

loca

[(38.9240081878896, -77.0235023040816),
 (38.9148887, -77.0045204),
 (38.9500972, -77.0778085),
 (38.89801985, -77.0480053952267),
 (38.913446, -77.0703076)]

In [7]:
geolocator = Nominatim(user_agent="high school")
loca=[]
for i in range(len(gh)):
    location = geolocator.geocode(gh["address"][i])
    loca.append((location.latitude, location.longitude))
    
gh["location"]=loca


In [ ]:
# locas=[]
# err=[]
# for i in range(len(sales)):
#     time.sleep(1)
#     try:
#         location = geolocator.geocode(sales["address"][i])
#         locas.append((location.latitude, location.longitude))
#     except:
#         locas.append((0,0))
#         err.append(sales.iloc[[i]])  
#         print(i)

In [ ]:
# sales["location"]=locas
# sales.to_csv("sales with address.csv")

In [15]:
sales=pd.read_csv("sales with address.csv",dtype={"Zip Code":str})

In [16]:
sales["location"]=sales["location"].map(lambda x:tuple(list(a)))

In [17]:
highsch=pd.DataFrame()
homes=pd.DataFrame()
for i in range(len(gh)):
    home=sales[sales["Zip Code"]==gh["Zip Code"][i]].reset_index()
    a=gh["location"][i]
    high=gh.loc[[i]]
    for j in range(len(home)):
        b=home["location"][j]
        home["distance"][j]=distance.distance(a, b).miles
    homes=pd.concat([homes,home])
    ave_beds=pd.DataFrame(home[home["distance"]<=1].groupby("Beds")["Current Price"].mean())
    ave_type=pd.DataFrame(home[home["distance"]<=1].groupby("Structure Type")["Current Price"].mean())
    for m in range(len(ave_beds)):
        aa=ave_beds.reset_index()
        name=str('%d Beds'%(aa.loc[m]["Beds"]))
        high[name]=[aa.loc[m]["Current Price"]]
    for n in range(len(ave_type)):
        bb=ave_type.reset_index()
        name2=str(bb.loc[n]["Structure Type"])
        high[name2]=[bb.loc[n]["Current Price"]]
    highsch=pd.concat([highsch,high])


C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [18]:
highsch

,# Grads,% Grad,0 Beds,1 Beds,2 Beds,3 Beds,4 Beds,5 Beds,6 Beds,7 Beds,...,School Code,School Name,Subgroup,Total Average,Twin/Semi-Detached,Unit/Flat/Apartment,Unnamed: 0,Zip Code,address,location
0,103,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,402,Benjamin Banneker HS,All,1109.0,NaN,NaN,0,20001,"800 Euclid St NW, Washington, DC 20001","(38.9240081878896, -77.0235023040816)"
1,154,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,466,School Without Walls HS,All,1272.0,NaN,NaN,13,20037,"151 T St NE, Washington, DC 20002","(38.9148887, -77.0045204)"
2,137,0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,458,McKinley Technology HS,All,1005.0,NaN,NaN,8,20002,"3950 Chesapeake St NW, Washington, DC 20016","(38.9501564, -77.0777757)"
3,65,0.98,NaN,391722.535714,539249.91954,7.913266e+05,9.924978e+05,1220625.0,1435000.0,NaN,...,478,"Phelps Architecture, Construction and Engineer...",All,875.0,6.928538e+05,449269.360656,17,20002,"2130 G St NW, Washington, DC 20037","(38.89801985, -77.0480053952267)"
4,118,1.00,200112.5,360688.080000,943500.00000,1.229389e+06,1.895678e+06,2798633.3,4372859.0,2470000.0,...,471,Duke Ellington School of the Arts,All,955.0,1.717682e+06,498891.877551,15,20007,"3500 R St NW, Washington, DC 20007","(38.913446, -77.0703076)"


In [19]:
highsch=highsch[['Unnamed: 0','School Name','School Code','Zip Code','address','location','Performance','Ranking',
                '0 Beds', '1 Beds', '2 Beds', '3 Beds', '4 Beds',
                '5 Beds', '6 Beds', '7 Beds','Detached', 'End of Row/Townhouse','Penthouse Unit/Flat/Apartment',
                 'Row/Townhouse','Twin/Semi-Detached', 'Unit/Flat/Apartment']]

In [21]:
highsch.to_csv("sales around high school.csv")

In [22]:
mid=pd.read_csv("good middle school.csv",dtype={"Dir_Zip":str})
ele=pd.read_csv("good elm sch.csv",dtype={"Dir_Zip":str})
mid["address"]=mid["Dir_Street"]+", "+mid["Dir_Zip"]
ele["address"]=ele["Dir_Street"]+", "+ele["Dir_Zip"]

In [23]:
geolocator = Nominatim(user_agent="mid school")

In [24]:
locas=[]
err=[]
for i in range(len(mid)):
    time.sleep(1)
    try:
        location = geolocator.geocode(mid["address"][i])
        locas.append((location.latitude, location.longitude))
    except:
        locas.append((0,0))
        err.append(mid.iloc[[i]])  
        print(i)
mid["location"]=locas

0


In [25]:
mid["location"][0]=(38.917247, -77.044335)

C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
mid

,Unnamed: 0,Dir_Name_Display,Dir_School_Code,Dir_Street,Dir_Zip,AYP_Status,AYP_Status_Desc,Energy_Star_Rtg,Enroll_Total,Prog_Growth_Math_Current_Year,...,Perf_Engage_Current_Year,Perf_Teachers_Current_Year,Performance of Growth,Performance of Safety,Performance of Engage,Performance of Teachers,Performance,Ranking,address,location
0,3,Oyster-Adams Bilingual School,292,2801 Calvert St. NW and 2020 19th St. NW,20008,Reward,Reward schools have the highest levels of stud...,58.0,674.0,0.60,...,0.91,0.94,5,5,4,4,4.50,5,"2801 Calvert St. NW and 2020 19th St. NW, 20008","(38.917247, -77.044335)"
1,10,School Without Walls @ Francis-Stevens,409,2425 N St. NW,20037,Rising,Rising schools need support to continue growth...,52.0,471.0,0.68,...,0.88,0.94,5,4,4,4,4.25,5,"2425 N St. NW, 20037","(38.9072319, -77.0434321)"
2,20,Whittier Education Campus,338,6201 5th St. NW,20011,Focus,Focus schools need targeted support to address...,71.0,341.0,0.67,...,0.98,0.93,5,4,5,3,4.25,5,"6201 5th St. NW, 20011","(38.9658569, -77.0192386)"
3,21,Capitol Hill Montessori School @ Logan,360,215 G Street NE,20002,Focus,Focus schools need targeted support to address...,90.0,361.0,0.79,...,0.94,0.89,5,5,5,2,4.25,5,"215 G Street NE, 20002","(38.8985204, -77.0025683027108)"
4,18,West Education Campus,336,1335 Farragut St. NW,20011,Rising,Rising schools need support to continue growth...,4.0,315.0,0.65,...,0.93,0.93,4,4,5,3,4.00,5,"1335 Farragut St. NW, 20011","(38.950883, -77.0310414064957)"


In [27]:
geolocator = Nominatim(user_agent="elm school")

In [28]:
from geopy.geocoders import Nominatim
from geopy import distance

In [29]:
locas=[]
err=[]
for i in range(len(ele)):
    time.sleep(1)
    try:
        location = geolocator.geocode(ele["address"][i])
        locas.append((location.latitude, location.longitude))
    except:
        locas.append((0,0))
        err.append(ele.iloc[[i]])  
        print(i)
ele["location"]=locas

In [30]:
midsch=pd.DataFrame()
homes=pd.DataFrame()
for i in range(len(mid)):
    home=sales[sales["Zip Code"]==mid["Dir_Zip"][i]].reset_index()
    a=mid["location"][i]
    midd=mid.loc[[i]]
    for j in range(len(home)):
        b=home["location"][j]
        home["distance"][j]=distance.distance(a, b).miles
    homes=pd.concat([homes,home])
    ave_beds=pd.DataFrame(home[home["distance"]<=1].groupby("Beds")["Current Price"].mean())
    ave_type=pd.DataFrame(home[home["distance"]<=1].groupby("Structure Type")["Current Price"].mean())
    for m in range(len(ave_beds)):
        aa=ave_beds.reset_index()
        name=str('%d Beds'%(aa.loc[m]["Beds"]))
        midd[name]=[aa.loc[m]["Current Price"]]
    for n in range(len(ave_type)):
        bb=ave_type.reset_index()
        name2=str(bb.loc[n]["Structure Type"])
        midd[name2]=[bb.loc[n]["Current Price"]]
    midsch=pd.concat([midsch,midd])

C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [31]:
midsch.columns

Index(['0 Beds', '1 Beds', '2 Beds', '3 Beds', '4 Beds', '5 Beds', '6 Beds',
       '9 Beds', 'AYP_Status', 'AYP_Status_Desc', 'Detached',
       'Dir_Name_Display', 'Dir_School_Code', 'Dir_Street', 'Dir_Zip',
       'End of Row/Townhouse', 'Energy_Star_Rtg', 'Enroll_Total',
       'Penthouse Unit/Flat/Apartment', 'Perf_Engage_Current_Year',
       'Perf_Growth', 'Perf_Safety', 'Perf_Teachers_Current_Year',
       'Performance', 'Performance of Engage', 'Performance of Growth',
       'Performance of Safety', 'Performance of Teachers',
       'Prog_Growth_Math_Current_Year', 'Prog_Growth_Read_Current_Year',
       'Ranking', 'Row/Townhouse', 'Safe_ISA_Current_Year',
       'Safe_Incidents_Current_Year', 'Safe_Safety_Current_Year',
       'Safe_Suspension_Current_Year', 'Safe_Truancy_Current_Year',
       'Twin/Semi-Detached', 'Unit/Flat/Apartment', 'Unnamed: 0', 'address',
       'location'],
      dtype='object')

In [32]:
midsch=midsch[['Dir_Name_Display', 'Dir_School_Code','Dir_Zip','address',
       'location','Performance','Ranking','0 Beds', '1 Beds', '2 Beds', '3 Beds', '4 Beds', '5 Beds', '6 Beds',
       '9 Beds','Detached','End of Row/Townhouse','Penthouse Unit/Flat/Apartment','Row/Townhouse', 
             'Twin/Semi-Detached', 'Unit/Flat/Apartment' ]]

In [35]:
midsch.to_csv("sales around middle school.csv")

In [36]:
ele

,Unnamed: 0,Dir_Name_Display,Dir_School_Code,Dir_Street,Dir_Zip,AYP_Status,AYP_Status_Desc,Energy_Star_Rtg,Enroll_Total,Prog_Growth_Math_Current_Year,...,Perf_Teacher,Perf_Engage_Current_Year,Performance of Growth,Performance of Safety,Performance of Teachers,Performance of Engage,Performance,Ranking,address,location
0,26,Lafayette Elementary School,261,5701 Broad Branch Rd. NW,20015,Reward,Reward schools have the highest levels of stud...,93.0,761,66%,...,0.96,0.97,5,5,5,5,5.00,5,"5701 Broad Branch Rd. NW, 20015","(38.9665705, -77.0679997)"
1,44,Ross Elementary School,305,1730 R St. NW,20009,Reward,Reward schools have the highest levels of stud...,41.0,171,85%,...,0.91,1.00,5,5,4,5,4.75,5,"1730 R St. NW, 20009","(38.9123678, -77.0397081267877)"
2,14,Eaton Elementary School,232,3301 Lowell St. NW,20008,Reward,Reward schools have the highest levels of stud...,31.0,477,70%,...,0.94,0.99,5,5,4,5,4.75,5,"3301 Lowell St. NW, 20008","(38.9327861, -77.0658191)"
3,52,Thomson Elementary School,326,1200 L St. NW,20005,Reward,Reward schools have the highest levels of stud...,48.0,314,75%,...,0.93,0.94,5,5,4,5,4.75,5,"1200 L St. NW, 20005","(38.903656, -77.02824)"
4,24,Key Elementary School,272,5001 Dana Pl. NW,20016,Reward,Reward schools have the highest levels of stud...,53.0,397,47%,...,0.96,0.96,4,5,5,5,4.75,5,"5001 Dana Pl. NW, 20016","(38.92686, -77.10063)"


In [37]:
elmsch=pd.DataFrame()
homes=pd.DataFrame()
for i in range(len(ele)):
    home=sales[sales["Zip Code"]==ele["Dir_Zip"][i]].reset_index()
    a=ele["location"][i]
    elm=ele.loc[[i]]
    for j in range(len(home)):
        b=home["location"][j]
        home["distance"][j]=distance.distance(a, b).miles
    homes=pd.concat([homes,home])
    ave_beds=pd.DataFrame(home[home["distance"]<=1].groupby("Beds")["Current Price"].mean())
    ave_type=pd.DataFrame(home[home["distance"]<=1].groupby("Structure Type")["Current Price"].mean())
    for m in range(len(ave_beds)):
        aa=ave_beds.reset_index()
        name=str('%d Beds'%(aa.loc[m]["Beds"]))
        elm[name]=[aa.loc[m]["Current Price"]]
    for n in range(len(ave_type)):
        bb=ave_type.reset_index()
        name2=str(bb.loc[n]["Structure Type"])
        elm[name2]=[bb.loc[n]["Current Price"]]
    elmsch=pd.concat([elmsch,elm])

C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [38]:
elmsch.columns

Index(['0 Beds', '1 Beds', '2 Beds', '3 Beds', '4 Beds', '5 Beds', '6 Beds',
       '7 Beds', '8 Beds', '9 Beds', 'AYP_Status', 'AYP_Status_Desc',
       'Detached', 'Dir_Name_Display', 'Dir_School_Code', 'Dir_Street',
       'Dir_Zip', 'End of Row/Townhouse', 'Energy_Star_Rtg', 'Enroll_Total',
       'Penthouse Unit/Flat/Apartment', 'Perf_Engage_Current_Year',
       'Perf_Growth', 'Perf_Safe', 'Perf_Teacher', 'Performance',
       'Performance of Engage', 'Performance of Growth',
       'Performance of Safety', 'Performance of Teachers',
       'Prog_Growth_Math_Current_Year', 'Prog_Growth_Read_Current_Year',
       'Ranking', 'Row/Townhouse', 'Safe_Attendance_District_Average',
       'Safe_ISA_Current_Year', 'Safe_Incidents_Current_Year',
       'Safe_Safety_Current_Year', 'Safe_Suspension_Current_Year',
       'Safe_Truancy_Current_Year', 'Twin/Semi-Detached',
       'Unit/Flat/Apartment', 'Unnamed: 0', 'address', 'location'],
      dtype='object')

In [39]:
elmsch=elmsch[['Dir_Name_Display', 'Dir_School_Code','Dir_Zip','address',
       'location','Performance','Ranking','0 Beds', '1 Beds', '2 Beds', '3 Beds', '4 Beds', '5 Beds', '6 Beds',
       '9 Beds','Detached','End of Row/Townhouse','Penthouse Unit/Flat/Apartment','Row/Townhouse', 
             'Twin/Semi-Detached', 'Unit/Flat/Apartment']]

In [40]:
elmsch.to_csv("sales around elm school.csv")

In [ ]:
mid